# Scrape and download PDFs from Ofgem website

In [10]:
from typing import List
from urllib import request
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import datetime
from datetime import date
from typing import List
from xmlrpc.client import Boolean
from selenium import webdriver
from pathlib import Path
import os
from contextlib import contextmanager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import staleness_of, StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import pdfminer
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
import io

TODAY_STR = date.today().strftime("%d%m%y")

In [2]:
# Define url to be scraped
URL = "https://www.ofgem.gov.uk/master-publications-library?sort=publication_date"
URL_list = []

# All links to be scraped
for page in range(1, 428):
    new_url = "https://www.ofgem.gov.uk/master-publications-library?sort=publication_date"+"&page="+str(page)
    URL_list.append(new_url)

In [3]:
# Define driver
driver = webdriver.Chrome("/Users/thomas/opt/anaconda3/envs/scraperenv/lib/python3.10/site-packages/chromedriver_autoinstaller/106/chromedriver")

In [4]:
# Scrape each link on given page

web_links = []

for url in URL_list:
    
    driver.get(url)
    time.sleep(5)

    links = []

    for num_links in range(1, 51):
        
        link = driver.find_elements_by_xpath(f'/html/body/div[3]/div/div/main/div[2]/div/div[2]/div/div/div[2]/div/div/div/div[2]/div/ol/li[{num_links}]/div/a')
        # link = WebDriverWait(driver, 20).until(lambda x: x.find_elements_by_xpath(f'/html/body/div[3]/div/div/main/div[2]/div/div[2]/div/div/div[2]/div/div/div/div[2]/div/ol/li[{num_links}]/div/a'))
        links.append(link)

    for href in links:
        for l in href:
            web_links.append(l.get_attribute('href'))


In [7]:
# See how many guidance documents there are
counter = 0
for ul in web_links:
    if "guidance" in ul:
        counter+=1
counter

680

In [9]:
# Access link if guidance contained in link and scrape pdf link from the link and download the pdf
counter = 0
for url in web_links:
    if "guidance" in url:
        driver.get(url)
        time.sleep(5)
        
        links = []
        
        link = driver.find_elements_by_xpath(f'/html/body/div[3]/div/div/main/div[2]/div/div[2]/div/div[1]/div/div[2]/div[2]/div/a')
        links.append(link)

        web_links = []
        for href in links:
            for l in href:
                web_links.append(l.get_attribute('href'))
        pdf_links = list(set([pdf for pdf in web_links if ".pdf" in pdf]))

        # Download the pdf
        for pdf in pdf_links:
            counter += 1
            response = requests.get(pdf)
            with open(os.path.join("/Users/thomas/Documents/BEIS/input_data/ofgem_pdfs2/" + str(counter) + ".pdf"), "wb") as file:
                file.write(response.content)



In [11]:
# Read pdfs from the directory and output to a dataframe
def read_pdf_from_directory(dir)-> pd.DataFrame:
    """
    Reads data from pdfs and adds a column in the df with the textual data
    """
    pdf_columns = []
    for pdf in os.listdir(dir):
        file = os.path.join(dir, pdf)
        try:
            i_f = open(file,'rb')
            resMgr = PDFResourceManager()
            retData = io.StringIO()
            txtConverter = TextConverter(resMgr,retData, laparams= LAParams())
            interpreter = PDFPageInterpreter(resMgr, txtConverter)
            for page in PDFPage.get_pages(i_f):
                interpreter.process_page(page)
            txt = retData.getvalue() 
            string_txt = str(txt)
            pdf_columns.append({"PDFs" : pdf, "Secondary Legislation" : string_txt}) 
        except:
            continue
    pdf_df = pd.DataFrame(pdf_columns)
    return pdf_df

pdfminer_df = read_pdf_from_directory("/Users/thomas/Documents/BEIS/input_data/ofgem_pdfs/")

The PDF <_io.BufferedReader name='/Users/thomas/Documents/BEIS/input_data/ofgem_pdfs2/412.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='/Users/thomas/Documents/BEIS/input_data/ofgem_pdfs2/416.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='/Users/thomas/Documents/BEIS/input_data/ofgem_pdfs2/414.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case
The PDF <_io.BufferedReader name='/Users/thomas/Documents/BEIS/input_data/ofgem_pdfs2/428.pdf'> contains a metadata field indicating that it should not allow te

In [14]:
# Output to dataframe
pdfminer_df.to_excel(f"/Users/thomas/Documents/BEIS/input_data/{TODAY_STR}-pdf_ofgem_rawdata.xlsx", engine='xlsxwriter')